## Deep Convolutional Generative Adversarial Networks (DCGANs)

Deep Convolutional Generative Adversarial Networks (DCGANs) combine the power of deep convolutional neural networks (CNNs) with adversarial training to generate high-quality images from random noise. DCGANs have applications in image synthesis, style transfer, and data augmentation. In this tutorial, we explore DCGAN architecture and implementation, focusing on the popular MNIST dataset for digit generation.

Import Tensorflow and other libraries

In [1]:
import tensorflow as tf
import glob
import imageio
import matplotlib.pyplot as plt
import numpy as np
import os
import PIL
from tensorflow.keras import layers
import time
from IPython import display

Download MNIST Dataset

In [2]:
(train_images, train_labels), (_, _) = tf.keras.datasets.mnist.load_data()

In [24]:
print("Length of train images = ", len(train_images))
print("Length of train labels = ", len(train_labels))
print("Train_mages shape = ", train_images.shape)
print("Data type = ", train_images.dtype)

Length of train images =  60000
Length of train labels =  60000
Train_mages shape =  (60000, 28, 28, 1)
Data type =  float32


Convolutional neural networks (CNNs) typically expect input data in the shape of [batch_size, height, width, channels]. Reshaping allows to rearrange the dimensions of the dataset to match the required input shape.

In [25]:
train_images = train_images.reshape(train_images.shape[0], 28, 28, 1) #.astype('float32')
print("Train_mages new shape = ", train_images.shape)
print("Data type = ", train_images.dtype)

Train_mages new shape =  (60000, 28, 28, 1)
Data type =  float32


 BUFFER_SIZE typically refers to the number of elements (samples) from the dataset that should be buffered (or shuffled) at a time. When you're dealing with large datasets that cannot fit entirely into memory, you can't shuffle the entire dataset at once. Instead, you shuffle a buffer of elements. Setting the BUFFER_SIZE parameter controls the size of this buffer. A larger buffer size allows for more effective shuffling but requires more memory.

 BATCH_SIZE refers to the number of samples that are propagated through the neural network in a single forward/backward pass. In other words, it's the number of samples processed before the model's parameters are updated. Using mini-batches instead of processing the entire dataset at once helps in reducing the memory usage and allows for faster training as computations can be parallelized across the batch.

In [32]:
BUFFER_SIZE = 60000
BATCH_SIZE = 256
# Batch and shuffle the data
train_dataset = tf.data.Dataset.from_tensor_slices(train_images).shuffle(BUFFER_SIZE).batch(BATCH_SIZE)
train_dataset

<_BatchDataset element_spec=TensorSpec(shape=(None, 28, 28, 1), dtype=tf.float32, name=None)>

tf.data.Dataset.from_tensor_slices(train_images): 
>This creates a TensorFlow dataset from the train_images data. The train_images variable is a tensor/array containing the training data. Each element of train_images is treated as a separate sample along the first dimension.

.shuffle(BUFFER_SIZE): 
> This shuffles the elements of the dataset with a buffer size of BUFFER_SIZE. Shuffling the dataset is important during training to prevent the model from learning spurious correlations due to the order of the training data. The BUFFER_SIZE parameter determines the number of elements from which the dataset is shuffled at once. A larger buffer size allows for more effective shuffling but requires more memory.

.batch(BATCH_SIZE): 
> This groups the elements of the dataset into batches of size BATCH_SIZE. Batching is done to improve computational efficiency by processing multiple samples in parallel. The BATCH_SIZE parameter specifies the number of elements in each batch.

### Create the models

#### The Generator

In [34]:
def make_generator_model():
    model = tf.keras.Sequential()
    model.add(layers.Dense(7*7*256, use_bias=False, input_shape=(100,)))
    model.add(layers.BatchNormalization())
    model.add(layers.LeakyReLU())

    model.add(layers.Reshape((7, 7, 256)))
    assert model.output_shape == (None, 7, 7, 256)  # Note: None is the batch size

    model.add(layers.Conv2DTranspose(128, (5, 5), strides=(1, 1), padding='same', use_bias=False))
    assert model.output_shape == (None, 7, 7, 128)
    model.add(layers.BatchNormalization())
    model.add(layers.LeakyReLU())

    model.add(layers.Conv2DTranspose(64, (5, 5), strides=(2, 2), padding='same', use_bias=False))
    assert model.output_shape == (None, 14, 14, 64)
    model.add(layers.BatchNormalization())
    model.add(layers.LeakyReLU())

    model.add(layers.Conv2DTranspose(1, (5, 5), strides=(2, 2), padding='same', use_bias=False, activation='tanh'))
    assert model.output_shape == (None, 28, 28, 1)

    return model